In [1]:
import pandas as pd

# 1. Clean data & Form new df with location information of Toronto

## 1.1 Get raw data from Wikipedia

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df=pd.read_html(url)[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 1.2 Drop cells with Borough that is "not assigned

In [3]:
df_drop=df[df["Borough"]!="Not assigned"] 
df_drop.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
# reset index
df_drop=df_drop.reset_index(drop=True)
df_drop.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 1.3 Merge cells that have the same Borough

In [13]:
df_merge=df_drop.groupby(["Postal Code","Borough"]).agg({"Neighborhood":lambda x:",".join(x)})
df_merge.head()

,,Neighborhood
Postal Code,Borough,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [14]:
df_merge=df_merge.reset_index(drop=False)
df_merge.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 1.4 Check if Neighborhood has “Nan” or "Not assigned"

In [15]:
empty_neighborhood = df_merge[df_merge['Neighborhood'].isna()].shape[0] #show how many rows in dataframe
empty_neighborhood

0

In [16]:
na_neighborhood=df_merge[df_merge["Neighborhood"]=="Not assigned"].shape[0]
na_neighborhood

0

## 1.5 Check the shape of dataframe

In [17]:
df_merge.shape

(103, 3)

## 1.6 Get the Latitude and Longitude of Toronto

In [18]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_0d14369c47114b6f95c11ce0a4f7e7dc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='p_hNcVgPL5-QiOqIpGEbLOn9_Ml5nBI23dBv6ZIec9e3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0d14369c47114b6f95c11ce0a4f7e7dc.get_object(Bucket='capstoneproject-donotdelete-pr-xe45nkm9516pmb',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

location = pd.read_csv(body)
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 1.7 Merge two tables

In [19]:
df_merge=df_merge.merge(location,how="inner",on="Postal Code")
df_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 1.8 Select Borough that contains ""Toronto

In [20]:
# in this project, only Boroughs contains "Toronto" are considered
df_merge["Borough"].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [21]:
df_merge=df_merge[df_merge["Borough"].isin(["East Toronto","Central Toronto","Downtown Toronto","West Toronto"])].reset_index(drop=True)
df_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
Toronto_data=pd.DataFrame(df_merge)

# 2. Creat map of Toronto

In [23]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

## 2.1 Get the latitude and longitude of Toronto

In [25]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown toronto are 43.6534817, -79.3839347.


## 2.2 Draw the map of Toronto

In [26]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# 3. Getting Venues Data using Foursquare

In [27]:
CLIENT_ID = '5MOJTEXGZNNE1TJL4EDCEARIBI1AZAVH0SNGXER3CNAB0O32' # your Foursquare ID
CLIENT_SECRET = 'S3H0AZPGUNPWF41AE3F24FIOQUNULNLR4N42QRW40F5EEFWX' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [28]:
# Creat function that can be used to find nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                latitudes=Toronto_data['Latitude'],
                                longitudes=Toronto_data['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [32]:
# check the size of toronto_venues
toronto_venues.shape

(1614, 7)

In [33]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [34]:
# check the number of unique venues
len(toronto_venues["Venue"].unique())

1053

In [35]:
# check the popular venue category
toronto_venues.groupby("Venue Category").count().sort_values(by="Neighborhood",ascending=False).head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Coffee Shop,142,142,142,142,142,142
Café,90,90,90,90,90,90
Restaurant,55,55,55,55,55,55
Italian Restaurant,42,42,42,42,42,42
Park,36,36,36,36,36,36
Hotel,33,33,33,33,33,33
Bakery,32,32,32,32,32,32
Japanese Restaurant,32,32,32,32,32,32
Bar,27,27,27,27,27,27


According to the above dataframe, we could see Italian restaurant,bakery, japanese restaurant, pizza place, sushi restaurant, seafood restaurant,etc.are popular restaurants in Toronto. What's more, Italian restaurnt are most marktable, however, it also indicates it is most competitive.Therefore, It is more suitable for beginners to choose a restaurants category that are popular but less competitive.

# 4. Analyze Venue distribution in each Neighborhood

In [36]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1614, 237)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
to_group=to_onehot.groupby(["Neighborhoods"]).mean().reset_index(drop=False)
to_group.head(10)

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.018519,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.052632
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.015873,0.0,0.0,0.015873,0.0,0.000000,0.0,0.015873
5,Christie,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
6,Church and Wellesley,0.012987,0.0000,0.0000,0.000,0.0000,0.000,0.012987,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.012987,0.0,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.0,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.010000,0.0,0.000000,0.0,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [43]:
sushi_nei=to_group[["Neighborhoods","Sushi Restaurant"]]
sushi_nei.head(10)

,Neighborhoods,Sushi Restaurant
0,Berczy Park,0.018519
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.015873
5,Christie,0.000000
6,Church and Wellesley,0.064935
7,"Commerce Court, Victoria Hotel",0.000000
8,Davisville,0.058824
9,Davisville North,0.000000


# 5. Cluster Neighborhoods by frequency of "Suchi Restaurant"

In [44]:
from sklearn.cluster import KMeans

In [45]:
# set number of clusters
toclusters = 3

to_clustering = sushi_nei.drop(["Neighborhoods"],1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 2, 0, 2, 0], dtype=int32)

In [46]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = sushi_nei.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged.head(10)

,Neighborhoods,Sushi Restaurant,Cluster Labels
0,Berczy Park,0.018519,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.015873,0
5,Christie,0.000000,0
6,Church and Wellesley,0.064935,2
7,"Commerce Court, Victoria Hotel",0.000000,0
8,Davisville,0.058824,2
9,Davisville North,0.000000,0


In [47]:
to_merged.rename(columns={"Neighborhoods":"Neighborhood"},inplace=True)

In [48]:
merge_loc=to_merged.merge(Toronto_data,on="Neighborhood",how="inner")
merge_loc.head(10)

,Neighborhood,Sushi Restaurant,Cluster Labels,Postal Code,Borough,Latitude,Longitude
0,Berczy Park,0.018519,0,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0,M6K,West Toronto,43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",0.000000,0,M7Y,East Toronto,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0,M5V,Downtown Toronto,43.628947,-79.394420
4,Central Bay Street,0.015873,0,M5G,Downtown Toronto,43.657952,-79.387383
5,Christie,0.000000,0,M6G,Downtown Toronto,43.669542,-79.422564
6,Church and Wellesley,0.064935,2,M4Y,Downtown Toronto,43.665860,-79.383160
7,"Commerce Court, Victoria Hotel",0.000000,0,M5L,Downtown Toronto,43.648198,-79.379817
8,Davisville,0.058824,2,M4S,Central Toronto,43.704324,-79.388790
9,Davisville North,0.000000,0,M4P,Central Toronto,43.712751,-79.390197


# 6. Visualize the clusters

In [49]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [50]:
# create map
map_cluster = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i + x + (i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merge_loc['Latitude'], merge_loc['Longitude'], merge_loc['Neighborhood'], merge_loc['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
    

       
map_cluster


In [51]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

NameError: name 'map_clusters' is not defined

# 7. Examine Clusters

In [52]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Sushi Restaurant,Cluster Labels
0,Berczy Park,0.018519,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.015873,0
5,Christie,0.000000,0
7,"Commerce Court, Victoria Hotel",0.000000,0
9,Davisville North,0.000000,0
10,"Dufferin, Dovercourt Village",0.000000,0
11,"First Canadian Place, Underground city",0.020000,0


In [131]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1].head(10)

,Neighborhood,Sushi Restaurant,Cluster Labels
12,"Forest Hill North & West, Forest Hill Road Park",0.25,1


In [132]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2].head()

,Neighborhood,Sushi Restaurant,Cluster Labels
6,Church and Wellesley,0.064935,2
8,Davisville,0.058824,2
16,"India Bazaar, The Beaches West",0.045455,2
23,"Queen's Park, Ontario Provincial Government",0.058824,2
28,"Runnymede, Swansea",0.058824,2


# Conclusion

Most sushi restaurants are in Cluster 2 which is around Wellesley, Davisville, and The Beaches West areas. Except for those areas, it's very hard to find a sushi restaurant. Therefore, by looking at the nearby venue, it looks like the area around Ossington, Dufferin, and Lansdowne are lack of a sushi restaurant. Therefore, this project recommends the entrepreneur to open a sushi restaurant in these locations with little or no competition but still have a market.

It should be noted that I only take into consideration of one factor: the distribution of sushi restaurants in each neighborhood in this project. In fact, factors should be taken into consideration such as residents’ preference, population density, the income of residents, rent that could influence the decision to open a new restaurant. Future research is needed to take into consideration these factors.
